In [1]:
import numpy as np
import pandas as pd
import datetime
import re
from dateutil.parser import parse
import math
import glob
import sys
import openpyxl as opx
import xlrd
import seaborn as sns
import random
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import font_manager, rc
# matplotlib.rcParams['axes.unicode_minus'] = False
# font_path = "C:/Windows/Fonts/malgun.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)
# %matplotlib inline


from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [2]:
# # 라이브러리 없다고 에러나올 시 아래 코드 실행 (뒤에 seaborn만 변경)
# pip install seaborn

SyntaxError: invalid syntax (334342175.py, line 2)

# data 추출

## 엑셀파일 concat

In [3]:
files = glob.glob('./raw_data/*.xlsx')

all_data = pd.DataFrame()

for f in files:
    data = pd.read_excel(f, parse_dates=True)    
    df = pd.DataFrame(data)    
    all_data = pd.concat([all_data, df])
    print(f, all_data.shape)
all_data


./raw_data\20230704.xlsx (184, 4)
./raw_data\20230705.xlsx (472, 4)
./raw_data\20230706.xlsx (760, 4)
./raw_data\20230707L.xlsx (1048, 4)
./raw_data\20230708.xlsx (1336, 4)
./raw_data\20230709.xlsx (1624, 4)
./raw_data\20230710.xlsx (1912, 4)
./raw_data\20230711.xlsx (2200, 4)
./raw_data\20230712.xlsx (2488, 4)
./raw_data\20230713.xlsx (2776, 4)
./raw_data\20230714.xlsx (3064, 4)
./raw_data\20230715.xlsx (3352, 4)
./raw_data\20230716.xlsx (3640, 4)
./raw_data\20230717.xlsx (3928, 4)
./raw_data\20230718.xlsx (4216, 4)
./raw_data\20230719.xlsx (4504, 4)
./raw_data\20230720.xlsx (4792, 4)
./raw_data\20230721.xlsx (5080, 4)
./raw_data\20230722.xlsx (5368, 4)
./raw_data\20230723.xlsx (5656, 4)
./raw_data\20230724.xlsx (5944, 4)
./raw_data\20230725.xlsx (6232, 4)
./raw_data\20230726.xlsx (6520, 4)
./raw_data\20230727.xlsx (6808, 4)
./raw_data\20230728.xlsx (7096, 4)
./raw_data\20230729.xlsx (7384, 4)
./raw_data\20230730.xlsx (7672, 4)
./raw_data\20230731.xlsx (7960, 4)
./raw_data\20230801.xl

NameError: name 'all_dat' is not defined

In [ ]:
df = all_data

In [ ]:
df = df.rename(columns={'Date': 'DateTime','16-bit Unsigned':'Power'})

## 인덱스 변경

In [ ]:
df['DateTime'] = pd.to_datetime(df['DateTime'])

In [ ]:
df = df.set_index('DateTime')

In [ ]:
df

## Time column drop

In [ ]:
df.drop(columns=['Time'], inplace=True)

In [ ]:
df

## CSV 파일로 export

In [ ]:
file_name = '230821.xlsx'
df.to_excel(file_name)

## 1차 visualization

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df[name])    
    plt.plot(df[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


# 엑셀데이터 불러오기

In [ ]:
df = pd.read_excel('230821.xlsx', parse_dates=True)  

In [ ]:
df

# 데이터 전처리

## 마스크 설정
- 650 이상이거나, 550 이하이면 outlier로 간주

In [ ]:
mask =  ((df['Millisecond']>= 650)|(df['Millisecond'] <= 550))

## outlier 추출

In [ ]:
df_outlier = df[mask]

In [ ]:
df_outlier

## Filtered 추출

In [ ]:
df_filtered = df[~mask]

In [ ]:
df_filtered

## Masking된 것 만 visualization

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df_filtered.columns)):
    plt.subplot(len(df_filtered.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df_filtered.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df_filtered[name])    
    plt.plot(df_filtered[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


## Outlier visulaization

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df_outlier.columns)):
    plt.subplot(len(df_outlier.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df_outlier.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df_outlier[name])    
    plt.plot(df_outlier[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


# Resampling

## 5분단위 평균

In [ ]:
df = df_filtered

In [ ]:
df2= df.resample('5T').mean()

In [ ]:
df2

In [ ]:
df3 = df2.interpolate(method='linear')

In [ ]:
df=df3

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df[name])    
    plt.plot(df[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


# Power가 0인값 초과인 경우만 추출

In [ ]:
df

In [ ]:
mask =  (df['Power']> 0)

In [ ]:
df_filtered = df[mask]

In [ ]:
df=df_filtered

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.plot(df[name])    
    plt.plot(df[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.scatter(df.index, df[name])    
    plt.scatter(df.index, df[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


## 5분단위 평균

In [ ]:
df = df.resample('5T').mean()

In [ ]:
df

In [ ]:
fig, ax = plt.subplots(figsize=(18,10))

for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.subplots_adjust(hspace=0.4)
    name = df.columns[i]
    r=random.random()
    g=random.random()
    b=random.random()
    color=(r,g,b)
    plt.scatter(df.index, df[name])    
    plt.scatter(df.index, df[name], color=color)    
    plt.title(name, loc = 'left', pad=15, fontsize=13, fontweight="bold")      
    plt.yticks(fontsize=13)    
    plt.xticks(fontsize=13)                
#     plt.xlim(pd.Timestamp('2022-04-20 00:00:00'), pd.Timestamp('2023-05-31 00:00:00'))    
    plt.grid(True)
plt.show()
fig.tight_layout()


## CSV 파일로 저장 

In [ ]:
df.to_csv('./treated_data/230821_Power_5min_average.csv')